In [1]:
######### Insert the following in CODE CELL 1

####################################
# This program is same as that defined in 'spam_not_spam_SVM.ipynb' the only
 # Difference is this program execute all functions using Button Events
#####################################

# spam detection in emails - Core Algorithm used is SVM, Accuracy above 97 %
#########################################################################

# Run the following commands only once, after mounting and connect
# Change Untitled.ipynb name to your acutal program name, then run the
# following.

#### Accuracy of SVM is 98 %, Notebook name is 'spam_not_spam_SVM', select this
#### Accuracy of NAIVEBAYES is 90 %
#### Accuracy of RANDOM FOREST is below 70 %

########### Start your program from here

# Why the !mv (Move Command)
# If you want to connect your actual program using another login program you should
# save all your programs in "/content/drive/MyDrive", folder

# By default colab store your program in folder '/content/drive/MyDrive/Colab Notebooks/'
# !mv command copies your program from the above folder to "/content/drive/MyDrive"

!mv "/content/drive/MyDrive/Colab Notebooks/spam_detection_SVM.ipynb" "/content/drive/MyDrive"

# Define function to set focus to result output grid, other wise you will have to manually
# scroll down to bottom of this cell to view results

from IPython.display import display, HTML,Javascript

def set_focus():
    display(Javascript("""
        document.getElementById('output-area').scrollIntoView({behavior: 'smooth'});
    """))

# Set focus to result grid
set_focus()

# Import color palettes

!pip install colorama
from colorama import Fore, Style, Back
import tqdm

# Install all necessary packages and libraries

!pip install pyspark
import pyspark

from matplotlib import pyplot as plt
from pyspark.ml.classification import LinearSVCModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, \
     VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, length, explode, col, when
from pyspark.ml.classification import LinearSVC

# Import the necessary libraries for user inputs

import ipywidgets as VBox
from IPython.display import display, HTML
from traitlets import TraitError
import csv
import ipywidgets as widgets
from IPython.display import display, HTML,Javascript

# Clear Result Grid package

from IPython.display import clear_output

# Create an instance of SparkSession, and store it in 'my_spark'. You can use any name
# other than 'my_spark'

my_spark = SparkSession.builder.getOrCreate()

mv: cannot stat '/content/drive/MyDrive/Colab Notebooks/spam_detection_SVM.ipynb': No such file or directory


<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=2d2cb4bd685a6ea79e40d5d7152ef18a4f6bfef8ecc8f2f67937186e7afd36d7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
######## Insert the following in CODE CELL 2

######################
# Important : To view output in Full Screen without showing codes,
# R.click Code cell-->'View output fullscreen'
#######################

# @title

# Set focus to result grid
set_focus()


# Function to clear the result grid

def clear_result_grid():
    clear_output(wait=True)

def load_data(event):
    print()
    clear_result_grid()
# global command is used to access df_from_csv dataframe in other functions,
# otherwise df_from_csv not declared error will flag by python
    global df_from_csv
    print()
    print()

    display(HTML("<h3 style='color:orange; text-align:left;'>Loading Data, this may take few seconds...wait !!!</h3>"))

    print()
    print()
# The .option("quote", "\"") specifies some email messages are inside quotation marks, so
# remove quotes and read it as a single mail

    df_from_csv = my_spark.read \
        .format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("quote", "\"") \
        .load("/content/drive/MyDrive/spam_naipu.csv")

    display(HTML("<h3 style='color:magenta; text-align:left;'>All your Data loaded Successfully...</h3>"))
    main_function()
def EDA_start(event):
    print()
    print()
#    clear_result_grid()
    display(HTML("<h3 style='color:orange; text-align:left;'>EDA Started, Please wait !!!</h3>"))

# Declare global df_from_csv again otherwise unbound error will be displayed in
# command, df_from_csv = df_from_csv.dropna(subset=['text'])

    global df_from_csv

    print("Schema of emails_dataset")
    df_from_csv.printSchema()

    print("Data in emails_dataset")
    df_from_csv.show(2)

# Display count of total 'ham' and 'spam' messages as follows:-

    tot_records = df_from_csv.count()
    ham_count = df_from_csv.filter(col('class') == 'ham').count()
    spam_count = df_from_csv.filter(col('class') == 'spam').count()

    not_ham_spam_count = df_from_csv.filter((col('class') != 'ham') &
     (col('class') != 'spam')).count()

    print("Total mails =", tot_records)
    print("Total ham mails =", ham_count)
    print("Total spam mails =", spam_count)

# Check for null values

    null_count = df_from_csv.filter(col("text").isNull()).count()
    print("Null values =", null_count)

    print("Total erroneous mails including nulls =", not_ham_spam_count)

# Findings :
#~~~~~~~~~~
# Total mails      = 5574
# Total ham mails  = 4825
# Total spam mails =  747

# Null values = 1
# Total erroneous mails including nulls = 2 (class column contain
# other than 'ham' or 'spam' category)

# Remove all erroneous rows, in this case 2 rows
# The 2 rows are removed in pre_process_data function below

###### Display average lengths of 'ham' and 'spam' messages

# To get avg, create a view and use simple SQL statement as follows

    df_from_csv.createOrReplaceTempView("avg_view")

# First group all 'ham' messages with Average, store it in avg-ham

    ham_msgs = my_spark.sql("SELECT AVG(LENGTH(text)) AS avg_ham FROM avg_view WHERE class = 'ham'")
    spam_msgs = my_spark.sql("SELECT AVG(LENGTH(text)) AS avg_spam FROM avg_view WHERE class = 'spam'")

    sum_ham_msgs = my_spark.sql("SELECT SUM(LENGTH(text)) AS sum_ham FROM avg_view WHERE class = 'ham'")
    sum_spam_msgs = my_spark.sql("SELECT SUM(LENGTH(text)) AS sum_spam FROM avg_view WHERE class = 'spam'")

    sum_ham_msgs_converted = round(sum_ham_msgs.first().sum_ham, 2)
    sum_spam_msgs_converted = round(sum_spam_msgs.first().sum_spam, 2)

    print("Total Characters in ham messages =", sum_ham_msgs_converted)
    print("Total Characters in spam messages =", sum_spam_msgs_converted)

# Retrieve average, show result using 2 decimal places

    ham_msg_length  = round(ham_msgs.first().avg_ham, 2)
    spam_msg_length = round(spam_msgs.first().avg_spam, 2)

    print("Average Length of ham message=", ham_msg_length)
    print("Average Length of spam message=", spam_msg_length)

# Average Length of ham message= 71.07 Characters
# Average Length of spam message= 138.46 Characters

# Findings : The above 'ham','spam' message lengths clearly idicates that, in the given
# ~~~~~~~~~~
# Dataset, spam messages are longer than ham messages. So include these lengths as a feature
# while train your model.

    display(HTML("<h3 style='color:magenta; text-align:left;'>EDA Operations finished...</h3>"))
    main_function()
def pre_process_data(event):
    print()
    clear_result_grid()
    display(HTML("<h3 style='color:orange; text-align:left;'>Pre-Processing of Data Started, Please wait !!!</h3>"))

# Declare 'global df_from_csv' again otherwise unbound error will be displayed in
# command, df_from_csv = df_from_csv.dropna(subset=['text'])

    global df_from_csv

#  Preprocess Stage 1: Remove all Null values from 'class' and 'text' columns

    df_from_csv = df_from_csv.dropna(subset=['class'])
    df_from_csv = df_from_csv.dropna(subset=['text'])

# Cross verify after removing null values

    null_count = df_from_csv.filter(col("text").isNull()).count()

    print("Null values =", null_count)

# Null values = 0, One null value found and removed

# Remove other erroneous rows

# Filter Only 'ham' and 'spam' form 'class' column of df_from_csv dataset

    df_from_csv = df_from_csv.filter((col('class') == 'ham') |
     (col('class') == 'spam'))

    print("Cross verify the filtered df_from_csv Dataset")
    df_from_csv.describe().show()

# The following count of class and text column shows correct number of emails as 5572

# +-------+-----+--------------------+
# |summary|class|                text|
# +-------+-----+--------------------+
# |  count| 5572|                5572|
# |   mean| null|               645.0|
# | stddev| null|                null|
# |    min|  ham| &lt;#&gt;  in mc...|
# |    max| spam|  _ we r stayin h..|
# +-------+-----+--------------------+


# Calculate Text Length of each mail and add it along with the original dataset
# text length plays a key role in predicting 'ham' or 'spam' mails. The reason is that,
# normaly spam messages contains more number of characters than ham mails

    global add_length_new

    add_length_col = df_from_csv.withColumn("text_length",
                                                length(df_from_csv["text"]))

    print("Schema of add_length_col Dataset...")
    add_length_col.printSchema()
# root
#  |-- class: string (nullable = true)
#  |-- text: string (nullable = true)
#  |-- text_length: integer (nullable = true) New column added

# Cross verify data in 'add_length_col' dataset

    print("Data in add_length_col DF...")
    add_length_col.show(5)

# +-----+--------------------+-----------+
# |class|                text|text_length|
# +-----+--------------------+-----------+
# | spam|XXXMobileMovieClu...|        149|
# |  ham|Oh k...i'm watchi...|         26|
# |  ham|Eh u remember how...|         81|
# |  ham|Fine if that  s t..|         58|
# | spam|England v Macedon...|        155| Spam Messages shows high text lengths
# +-----+--------------------+-----------+

# Remove zero length documents

    add_length_col.createOrReplaceTempView("drop_zero_docs")

    remove_0_docs = my_spark.sql("SELECT * FROM drop_zero_docs where text_length > 0")

    remove_0_docs.createOrReplaceTempView("clean_class_view")

    add_length_new = my_spark.sql("SELECT * FROM clean_class_view WHERE class = 'ham' OR class = 'spam'")

    print("Class column cleaned add_length_new")
    add_length_new.describe().show()

    print("Average lenths of ham and spam messages are....")
    add_length_new.groupby("class").mean().show()
# +-----+------------------+
# |class|  avg(text_length)|
# +-----+------------------+
# |  ham|  71.0334510700187|
# | spam|138.31935047361299|
# +-----+------------------+

    print("Schema of add_length_new")
    add_length_new.printSchema()

# root
#  |-- class: string (nullable = true)
#  |-- text: string (nullable = true)
#  |-- text_length: integer (nullable = true) this column is added in actual dataset

    display(HTML("<h3 style='color:magenta; text-align:left;'>Pre-Processing Finished...</h3>"))
    main_function()
def transformation(event):
    print()
    set_focus()
    clear_result_grid()
    display(HTML("<h3 style='color:orange; text-align:left;'> \
      Transformation Started may take less than 2 Minutes, Please wait !!!</h3>"))

# Create an Instance of Tokenizer as tokenizer, to split each words separately

    tokenizer = Tokenizer(inputCol="text", outputCol="text_tokenized")

# Apply Tokenizer into add_length_col DF, to get each word separately

    tokenized_words_df = tokenizer.transform(add_length_new)

    print("Schema of tokenized_words_df dataframe...")
    tokenized_words_df.printSchema()
# root
#  |-- class: string (nullable = true)
#  |-- text: string (nullable = true)
#  |-- text_length: integer (nullable = true)
#  |-- text_tokenized: array (nullable = true)  new column added, contains each word
#  |    |-- element: string (containsNull = true)

    print("Display tokenized_words_df dataframe...")
    tokenized_words_df.show(5, False, vertical=True)
# -RECORD 0------------------------------------------------------------------------------
#  class          | spam
#  text           | XXXMobileMovieClub: To use your credit, click the WAP link in the next...
#  text_length    | 149
#  text_tokenized | [xxxmobilemovieclub:, to, use, your, credit,, click, the, wap, link, in...

# Remove Stop words as follows, 'is','you','i','and','we'..etc:-

    stop_words_grouping = StopWordsRemover(inputCol="text_tokenized", outputCol="text_with_no_stopwords")

#    stop_words_removed = stop_words_grouping.transform(tokenizer.transform(add_length_col))

    stop_words_removed = stop_words_grouping.transform(tokenized_words_df)

    print("Stop words removed DF, stop_words_removed...")
    stop_words_removed.show(5, False, vertical=True)

# Count total number of indivudual words in the whole datast

    stop_words_exploded = stop_words_removed.select(explode("text_with_no_stopwords"))
    total_words = stop_words_exploded.count()
    print("Total number of words after removing Stop words:", total_words)


    total_words_before = stop_words_removed.select(explode("text_tokenized"))
    total_words_before_SWR = total_words_before.count()

    print("Total number of words in the whole Document:", total_words_before_SWR)

# Total number of words in the whole Document: 86348

# In PySpark, CountVectorizer is a machine learning feature extraction tool that is
# used to convert a collection of text documents into a matrix of token counts.
# The CountVectorizer function takes a sequence of text documents as input and
# produces a sparse matrix of token counts as output.

# In other words CountVectorizer gives a value to each words in a single document
# that is, if 'A=seb is teaching java' , seb = 1 / 4, seb appears only once in
# document A which contains 4 words

# A sparse matrix is a matrix in which most of the elements are zero. In other words,
# it is a matrix that has a large number of zero elements compared to the total number of elements.

# Important : The parameter 'vocabSize' is mandatory while declaring CountVectorizer, otherwise when
# you give user emails into your model will throw 'stage failure' and 'Index out of range' errors

# How to determine the vocabSize ? Take 10 percent of total words in the whole document,
# In the origianl dataset nearly 86,000 words are found. But the distinct words will
# come around 80,0000. So 10 % of 80,000 will be 8000

    count_vector = CountVectorizer(inputCol='text_tokenized',
                                   outputCol='count_vectors', vocabSize=8000)

# fit the CountVectorizer to the data

    my_model = count_vector.fit(tokenized_words_df)

# transform the data using the CountVectorizer

    result = my_model.transform(tokenized_words_df)

    print("Data In result DF, after count vector function is applied...")
    result.show(5, False, vertical=True)

# How to Interpret the result DF, count_vectors colmn

# The first element of the tuple, 8000, represents the size of the vector, or total features
# i.e., the total number of elements in the vector.

# The second element of the tuple, [0,4,8,11,24,81,188,198,268,1202,1379,1400,7273],
# represents the indices of the non-zero elements in the vector. In this case, the non-zero
# elements are at positions [0,4,8,11,24,81,188.....] etc

# -RECORD 0------------------------------------------------------------------------------------------
#  class          | spam
#  text           | XXXMobileMovieClub: To use your credit, click the WAP link in the next txt .....
#  text_length    | 149
#  text_tokenized | [xxxmobilemovieclub:, to, use, your, credit,, click, the, wap, link, in, the,...
#  count_vectors  | (8000,[0,4,8,11,24,81,188,198,268,1202,1379,1400,7273],[1.0,2.0,1.0,1.0,1.......
#

# Calculate Inverse document frequency of words(IDF)
#  IDF is used to determine the importance of a term (word) in a corpus (collection of documents)
# that is if 'A=seb is teaching java', 'B = seb is teaching spark'
# seb = log(2 / 2). Here we have a corpus contains 2 documents
# so 'seb' appears only twice in 2 documents(corpus)

    tf_idf = IDF(inputCol="count_vectors", outputCol="tf_idf")

    tf_idf_model = tf_idf.fit(result)

# Transform tf_idf_model with original data, here in tf_idf_transformed dataframe
# the .transform(reuslt), tf and idf values are multiplied together

    tf_idf_transformed = tf_idf_model.transform(result)

    print("transformed tf_idf_model...")

    tf_idf_transformed.printSchema()

    inputs = ["tf_idf", "text_length"]
    output = "features"

    ham_spam_assembler = VectorAssembler(inputCols= inputs , outputCol= output)

# Create Vectorized Features using tf-Idf dataframe. tf_idf is transformed from tokenized_words
# data frame

    feature_df = ham_spam_assembler.transform(tf_idf_transformed)

# Cross Verify

    print("Schema of feature_df")
    feature_df.printSchema()

# Below schemat tf_idf column contains products of term frequency and inverse document frequencies
# in vector format

# features column contains tf * idf values along with text length of each document

# root
#  |-- class: string (nullable = true)
#  |-- text: string (nullable = true)
#  |-- text_length: integer (nullable = true)
#  |-- text_tokenized: array (nullable = true)
#  |    |-- element: string (containsNull = true)
#  |-- count_vectors: vector (nullable = true)
#  |-- tf_idf: vector (nullable = true)
#  |-- features: vector (nullable = true)

# Data in feature_df

    print("Data in feature_df")
    feature_df.show(5, False, vertical=True)

# Finding :  features column shows 8001 vectors, because along with tf_idf dataframe
# text_length is also added

# -RECORD 0------------------------------------------------------------------------------------------
# class          | spam
# text           | XXXMobileMovieClub: To use your credit, click the WAP link in the next txt message ...
# text_length    | 149
# text_tokenized | [xxxmobilemovieclub:, to, use, your, credit,, click, the, wap, link, in, the, next....
# count_vectors  | (8000,[0,4,8,11,24,81,188,198,268,1202,1379,1400,7273],[1.0,2.0,1.0,1.0,1.0,1.0,1.0...
# tf_idf         | (8000,[0,4,8,11,24,81,188,198,268,1202,1379,1400,7273],[1.2069184916283124,......)
# features       | (8001,[0,4,8,11,24,81,188,198,268,1202,1379,1400,7273,8000.......,149]

# Create an indexer for class column to Add label vector

    ham_spam_indexer = StringIndexer(inputCol='class', outputCol='label')

# Add the above label column with feature_df Dataframe

    label_df = ham_spam_indexer.fit(feature_df).transform(feature_df)

    print("Schema of label-df....")
    label_df.printSchema()

# root
#  |-- class: string (nullable = true)
#  |-- text: string (nullable = true)
#  |-- text_length: integer (nullable = true)
#  |-- text_tokenized: array (nullable = true)
#  |    |-- element: string (containsNull = true)
#  |-- count_vectors: vector (nullable = true)
#  |-- tf_idf: vector (nullable = true)
#  |-- features: vector (nullable = true)
#  |-- label: double (nullable = false)

    print("Display data in label_df..")
    label_df.show(5, False, vertical=True)

#     ############### Split Training and Test Data begins here ##################

# Cross Verify : count total number of documents or sentences in label_df as follows:-

    total_docs = label_df.count()
    print("Total number of documents...", total_docs)

# Total number of documents... 5552, Tallied with Total Rows after cleaning

# Split 70 % for training and 30 % for testing

    train_docs = int(total_docs * 0.70)

    test_docs = total_docs - train_docs

# Cross Verify train_docs and test_docs count

    print("Total Train documents = ", train_docs)
    print("Total Test documents = ", test_docs)

# Cross Verify

# Total number of documents... 5552
# Total Train documents =  3886
# Total Test documents =  1666

###### Findings : If Random split not showing correct results then manually extract
# training data and test data as follows. Otherwise you can proceed with Random split
######################

#     Extract 2935 documents or rows from label_df
#
#     train_data_df = label_df.limit(train_docs)
#
#     Similarly extract the remaining 1259 rows from label_df, for testing
#
#     test_data_df = label_df.subtract(train_data_df).limit(test_docs)

############ random split starts here

    train_data_df, test_data_df = label_df.randomSplit([0.7, 0.3], seed=5043)

    print("Schema of test_data_df")
    test_data_df.printSchema()

    print("Data in Test Data DF")
    test_data_df.show(2, False, vertical=True)


################# Create an SVM model object starts here ##########

# maxIter is, A maxIter value of 10 means that the algorithm will perform up to 10
# iterations to optimize the model. If the algorithm converges before reaching the maximum
# number of iterations, it will stop early. If the algorithm has not converged after maxIter
# iterations, it will stop and return the current best solution.

# regmParam=0.1, In PySpark's implementation of Support Vector Machines (SVMs), the regParam parameter
# is a regularization parameter that controls the amount of regularization applied to the model.
# Regularization is a technique used to prevent overfitting by adding a penalty term to the
# loss function that the model is trying to optimize.

    svm = LinearSVC(maxIter=10, regParam=0.1, featuresCol='features', labelCol='label')

    svm_model = svm.fit(train_data_df)
    prediction_df = svm_model.transform(test_data_df)

# Cross Verify Prediction Results

    print("Predictions are prediction_df...")
    prediction_df.show(5, False, vertical=True)

#
#     ##################### Important : Explain DAG : Directed Asyclic Graph
#     #  DAG is a directed graph that contains no cycles. This means that it is a graph where
#     #  you can only move from one vertex to another in one direction, and you cannot get back
#     #  to the starting point by following the edges. DAGs are used to represent many types of
#     #  data and processes in computer science, including data dependencies, task scheduling, and more.
#     ####################

# Prediction Accuracy of Model
#     # For multiclass classification problems in PySpark, you can use other
#     # evaluation metrics such as 'accuracy', 'weightedPrecision', 'weightedRecall', 'f1',
#     # 'weightedFMeasure', 'microAvgFMeasure', 'macroAvgFMeasure', 'weightedFalsePositiveRate',
#     # 'weightedTruePositiveRate'
#
#     # Important : Here MulticlassClassificationEvaluator is used because
#     #           : BinaryClassificationEvaluator shows 0.0 percent accuracy where a
#     #           : the scatter plot gives correct accuracy. So change it to
#     #           : MulticlassClassificationEvaluator, as follows:-
#
    evaluator = MulticlassClassificationEvaluator() \
        .setLabelCol("label") \
        .setPredictionCol("prediction") \
        .setMetricName("accuracy")

    prediction_accuracy = evaluator.evaluate(prediction_df)

    print("Prediction Accuracy of SVM Model =", prediction_accuracy * 100)

#     # Plot a Graph

    labels = prediction_df.select("label").toPandas()
    predictions = prediction_df.select("prediction").toPandas()

    plt.xlabel("Predictions")

    plt.ylabel("label")

    plt.scatter(predictions, labels)
    plt.show()

#     # Save Your Model as follows:-

    print()
    print()

    svm_model.write().overwrite() \
        .save("/content/drive/MyDrive/my_model_SVM")
    print("Your Model is saved in spark warehouse...as email_spam_model_rf")

    print("feature_df Schema")
    feature_df.printSchema()

# Cross Verification : Check the number of features in the training and test data

    print("Number of features in training data:", svm_model.numFeatures)
    print("Number of features in test data:", test_data_df.select("features").take(1)[0].features.size)


    display(HTML("<h3 style='color:magenta; text-align:left;'>Transformation Finished...</h3>"))
    main_function()

def load_model_and_predict(event):
    print()
    clear_result_grid()
    display(HTML("<h3 style='color:orange; text-align:left;'>Prediction Process started, Please wait !!!</h3>"))

    loaded_model = LinearSVCModel.load("/content/drive/MyDrive/my_model_SVM")

# Open the file in the original encoding (e.g. ANSI)

    user_df_csv = my_spark.read \
        .format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("quote", "\"") \
        .load("/content/drive/MyDrive/user_emails.csv")

# Merge or Append the actual dataset 'spam_naipu.csv' with user inputs using union command
# This operation is performed because in the saved model there are 8000 feature columns
# where as in the end user input emails total number of feature columns will be less than
# 8000. In this case pyspark throws 'stage failure errors'.

    original_df = my_spark.read \
        .format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("quote", "\"") \
        .load("/content/drive/MyDrive/spam_naipu.csv")

# Combine original dataframe with user_df, 'union' is used for this


    user_df_csv = original_df.union(user_df_csv)


# # Remove zero length documents

    add_length_col_user = user_df_csv.withColumn("text_length",
                                             length(user_df_csv["text"]))

    add_length_col_user.createOrReplaceTempView("drop_zero_docs")

    remove_0_docs = my_spark.sql("SELECT * FROM drop_zero_docs where text_length > 0")

    remove_0_docs.createOrReplaceTempView("clean_class_view")

    add_length_new_user = my_spark.sql("SELECT * FROM clean_class_view WHERE class = 'ham' OR class = 'spam'")

    # print("Statistics of remove_0_lenthg documents")
    # remove_0_docs.describe().show()

# Split each words and Tokenize it

    tokenizer = Tokenizer(inputCol="text", outputCol="text_tokenized")
    tokenized_words_df_user = tokenizer.transform(remove_0_docs)

    stop_words_grouping = StopWordsRemover(inputCol="text_tokenized", outputCol="text_with_no_stopwords")

    stop_words_removed_user = stop_words_grouping.transform(tokenized_words_df_user)

    # print("stop_words_removed_user")
    # stop_words_removed_user.show(2, False, vertical=True)

    count_vector_user= CountVectorizer(inputCol="text_tokenized",
                                       outputCol="count_vectors", vocabSize=8000)

#  # fit the CountVectorizer to the data

    my_model_user = count_vector_user.fit(stop_words_removed_user)

# transform the data using the CountVectorizer

    result_user = my_model_user.transform(stop_words_removed_user)

    tf_idf_user = IDF(inputCol="count_vectors", outputCol="tf_idf")
    tf_idf_model = tf_idf_user.fit(result_user)

#  # Transform tf_idf_model with original data

    tf_idf_transformed_user = tf_idf_model.transform(result_user)

    inputs = ["tf_idf", "text_length"]
    output = "features"

    ham_spam_assembler = VectorAssembler(inputCols=inputs, outputCol=output)
    feature_df_user = ham_spam_assembler.transform(tf_idf_transformed_user)

# Cross Verify - Important
# Total number of features in original dataset is 8001, including text_length
# This should tally with the user_emails dataset, otherwise 'Stage failure' and
# 'index out of bounds or range error will display

##### Start From here

# Count the number of non-zero elements in the "features" column
############
#     # Select the "features" column from the DataFrame
#     existing_features = feature_df_user.select("features")
#     print("existing features columns in feature_df_user are...")
#     existing_features.show(5)
#     missing_features = [f'feature{i}' for i in range(num_features, 8000)]
#     #
#     # zeros = [0 for _ in range(len(feature_df_user.select("features").first()))]
#     #
#     feature_df_user = concat(feature_df_user["features"], * missing_features)
#     print("feature_df_user Schema")
#     feature_df_user.printSchema()
# #    feature_df_user = feature_df_user.select("features")
#
# # Fill in any missing values with zeros
# # Replace null or missing values with zeros
#
#     feature_df_user = feature_df_user.select([when(feature_df_user[col].isNull(), 0.0).otherwise(feature_df_user[col]).alias(col) for col in feature_df_user.columns])

# Cross Verify - Important
# Total number of features in original dataset is 8001, including text_length
# This should tally with the user_emails dataset, otherwise 'Stage failure' and
# 'index out of bounds or range error will display

    # print("Number of features in user test data:",
    #       feature_df_user.select("features").take(1)[0].features.size)

############ Actual prediction using user data Start from here

    prediction_df_user = loaded_model.transform(feature_df_user)

# Sometimes ALS algorithm generates null values, so drop those null values

    prediction_df_user = prediction_df_user.dropna()
#

#
#     prediction_df_user.show(5, False, vertical=True)
#
# Create a new column that contains the first 3 characters of the text column(test)
# The user can input actual email as follows :-
#
# aks stands for short form of akshaya
#
# aks,test message typed today,,,
# aks,test taken from seb gmail spam You've won! please confirm receipt Now Get your
# aks,test Ozwin -Bonus? Congratulations! You are the chosen one!e!,,,
# aks,test message typed again manually,,,
#
    prediction_df_user  = prediction_df_user.withColumn('first_four_chars', substring('text', 1, 4))

#
# Re-Convert labels back to ham or spam (For prediction ham=0.0, and spam=1.0)

    prediction_df_user_converted = prediction_df_user\
         .withColumn("prediction_converted", when(prediction_df_user.prediction == 0.0, "ham")
                                       .when(prediction_df_user.prediction  == 1.0, "spam")
                                       .otherwise('Error'))

# Display prediction results of user mails start with 'test'
#
    prediction_df_user_converted.createOrReplaceTempView("last_view")

    my_spark.sql("SELECT text, prediction,prediction_converted FROM "
                  "last_view WHERE first_four_chars = 'test'")\
         .show(20, False, vertical=True)
    display(HTML("<h3 style='color:magenta; text-align:left;'>Prediction Finished...</h3>"))
    main_function()
# The following function predict single emails

def input_email_and_predict(event):
    print()
    clear_result_grid()
    filename = '/content/drive/MyDrive/user_input_mail.csv'

    headers = ['class','text']

    css = """
      <style>
          .red-text .widget-label {
              color: red !important;
              font-size: 18px;
          }
      </style>
      """
    display(HTML(css))
    print()
    print()

# Display Main Heading

    display(HTML("<h2 style='color:green; text-align:center;'>Final Prediction using new Emails</h2>"))
    print()

# Actual GUI descriptions for each required entry parameters declared here

    user_class = widgets.Dropdown(options = ['xxx,'],
                                  description ='Input not allowed here>>>>>>>>',
                                  style = {'description_width': 'initial'},
                                  layout=widgets.Layout(width='350px'))

    user_mail = widgets.Textarea(value='test',description='Copy and Paste your email>>>>',
                                    style={'description_width': 'initial'},
                                    layout=widgets.Layout(width='1000px',height='200px'))

# New description below the Textarea
    # new_description = widgets.Label("Note: The word 'test' cannot be removed from the beginning."
    # "Paste your Email after 'test'")

    new_description = widgets.HTML(
    "<p style='color: BLUE;'>Note: The word 'test' cannot be removed from the beginning. "
    "Paste your Email after 'test'</p>"
     )
# Add CSS classes declared above to the input widgets

    user_class.add_class('red-text')
    user_class.add_class('custom-input-box')
    user_class_container = widgets.HBox(children=[user_class])


    user_mail.add_class('red-text')
    user_mail.add_class('custom-input-box')

    user_mail_container = widgets.VBox(children=[user_mail,new_description])

# Create a Save or Submit button

    submit_button = widgets.Button(description='Save', style={'button_color': 'yellow'})

# Set the size of the button using Layout object

    button_layout = widgets.Layout(width='200px', height='50px', grid_area='center')
    submit_button.layout = button_layout

# Create Predict button

    button_predict = widgets.Button(description='Predict', style={'button_color': 'yellow'})
    button_predict.layout= button_layout

    buttons_container = widgets.HBox(children=[submit_button, button_predict])

#    display(user_class_container)

    display(user_mail_container)

# Apply CSS style to center the buttons

    centered_style = """
    display: flex;
    justify-content: center;
    """
    buttons_container.add_class('centered-buttons')
    display(HTML(f'<style>.centered-buttons{{{centered_style}}}</style>'))
    display(buttons_container)
    def on_button_clicked(submit_button):
# Assign values from text boxes to variables
      entered_class = user_class.value
      entered_mail = user_mail.value
# Save your data as csv file 'user_input_house.csv'
      with open(filename, 'w', newline='') as f:
              writer = csv.writer(f)
              writer.writerow(headers)
              writer.writerow([entered_class, entered_mail])
# $$ modify print statement
              print('Data has been written to', filename)

    submit_button.on_click(on_button_clicked)
    button_predict.on_click(predict_function)
    return True
def predict_function(parameter):

    # display(HTML("<h3 style='color:green; text-align:center;'>Prediction Process Single Mail started..may take few seconds !!!</h3>"))
    # print()
# Transfer Control from GUI to Colab Notebook function
    load_model_single_email()
    return

def load_model_single_email():
    print()
    clear_result_grid()
    display(HTML("<h3 style='color:orange; text-align:left;'>Prediction Process single "
      "Mail started, Please wait !!!</h3>"))

    loaded_model = LinearSVCModel.load("/content/drive/MyDrive/my_model_SVM")

# Open the file in the original encoding (e.g. ANSI)

    user_df_csv = my_spark.read \
        .format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("quote", "\"") \
        .load("/content/drive/MyDrive/user_input_mail.csv")

# Merge or Append the actual dataset 'spam_naipu.csv' with user inputs using union command
# This operation is performed because in the saved model there are 8000 feature columns
# where as in the end user input emails total number of feature columns will be less than
# 8000. In this case pyspark throws 'stage failure errors'.

    original_df = my_spark.read \
        .format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .option("quote", "\"") \
        .load("/content/drive/MyDrive/spam_naipu.csv")

# Combine original dataframe with user_df, 'union' is used for this

    user_df_csv = original_df.union(user_df_csv)

# # Remove zero length documents

    add_length_col_user = user_df_csv.withColumn("text_length",
                                             length(user_df_csv["text"]))

    add_length_col_user.createOrReplaceTempView("drop_zero_docs")

    remove_0_docs = my_spark.sql("SELECT * FROM drop_zero_docs where text_length > 0")

    remove_0_docs.createOrReplaceTempView("clean_class_view")

    add_length_new_user = my_spark.sql("SELECT * FROM clean_class_view WHERE class = 'ham' OR class = 'spam'")

    # print("Statistics of remove_0_lenthg documents")
    # remove_0_docs.describe().show()

# Split each words and Tokenize it

    tokenizer = Tokenizer(inputCol="text", outputCol="text_tokenized")
    tokenized_words_df_user = tokenizer.transform(remove_0_docs)

    stop_words_grouping = StopWordsRemover(inputCol="text_tokenized", outputCol="text_with_no_stopwords")

    stop_words_removed_user = stop_words_grouping.transform(tokenized_words_df_user)

    # print("stop_words_removed_user")
    # stop_words_removed_user.show(2, False, vertical=True)

    count_vector_user= CountVectorizer(inputCol="text_tokenized",
                                       outputCol="count_vectors", vocabSize=8000)

#  # fit the CountVectorizer to the data

    my_model_user = count_vector_user.fit(stop_words_removed_user)

# transform the data using the CountVectorizer

    result_user = my_model_user.transform(stop_words_removed_user)

    tf_idf_user = IDF(inputCol="count_vectors", outputCol="tf_idf")
    tf_idf_model = tf_idf_user.fit(result_user)

#  # Transform tf_idf_model with original data

    tf_idf_transformed_user = tf_idf_model.transform(result_user)

    inputs = ["tf_idf", "text_length"]
    output = "features"

    ham_spam_assembler = VectorAssembler(inputCols=inputs, outputCol=output)
    feature_df_user = ham_spam_assembler.transform(tf_idf_transformed_user)

# Cross Verify - Important
# Total number of features in original dataset is 8001, including text_length
# This should tally with the user_emails dataset, otherwise 'Stage failure' and
# 'index out of bounds or range error will display

##### Start From here

# Count the number of non-zero elements in the "features" column
############
#     # Select the "features" column from the DataFrame
#     existing_features = feature_df_user.select("features")
#     print("existing features columns in feature_df_user are...")
#     existing_features.show(5)
#     missing_features = [f'feature{i}' for i in range(num_features, 8000)]
#     #
#     # zeros = [0 for _ in range(len(feature_df_user.select("features").first()))]
#     #
#     feature_df_user = concat(feature_df_user["features"], * missing_features)
#     print("feature_df_user Schema")
#     feature_df_user.printSchema()
# #    feature_df_user = feature_df_user.select("features")
#
# # Fill in any missing values with zeros
# # Replace null or missing values with zeros
#
#     feature_df_user = feature_df_user.select([when(feature_df_user[col].isNull(), 0.0).otherwise(feature_df_user[col]).alias(col) for col in feature_df_user.columns])

# Cross Verify - Important
# Total number of features in original dataset is 8001, including text_length
# This should tally with the user_emails dataset, otherwise 'Stage failure' and
# 'index out of bounds or range error will display

    # print("Number of features in user test data:",
    #       feature_df_user.select("features").take(1)[0].features.size)

############ Actual prediction using user data Start from here

    prediction_df_user = loaded_model.transform(feature_df_user)

# Sometimes ALS algorithm generates null values, so drop those null values

    prediction_df_user = prediction_df_user.dropna()
#

    prediction_df_user  = prediction_df_user.withColumn('first_four_chars', substring('text', 1, 4))

# Re-Convert labels back to ham or spam (For prediction ham=0.0, and spam=1.0)

    prediction_df_user_converted = prediction_df_user\
         .withColumn("prediction_converted", when(prediction_df_user.prediction == 0.0, "ham")
                                       .when(prediction_df_user.prediction  == 1.0, "spam")
                                       .otherwise('Error'))

# Display prediction results of user mails start with 'test'

    prediction_df_user_converted.createOrReplaceTempView("last_view")

    # my_spark.sql("SELECT text, prediction,prediction_converted FROM "
    #               "last_view WHERE first_four_chars = 'test'")\
    #      .show(20, False, vertical=True)

# By using triple quotes("""), you can write the entire SQL query as a single string,
# even if it spans multiple lines

    html_table = my_spark.sql("""
    SELECT prediction_converted,
           CASE WHEN prediction_converted = 'ham'
           THEN '<h2 style="font-size: 24px;color:green;">Not Spam mail !!!</h2>'
                WHEN prediction_converted = 'spam'
           THEN '<h2 style="font-size: 24px;color:red;">SPAM mail !!!</h2>'
           ELSE prediction_converted
                END AS prediction_converted_html
    FROM last_view
    WHERE first_four_chars = 'test' AND (prediction_converted = 'ham' OR prediction_converted = 'spam')
    """)

# Convert the result to Pandas DataFrame
    pandas_df = html_table.toPandas()

# Generate HTML content
    html_content = pandas_df[["prediction_converted_html"]].to_html(index=False, header=False, escape=False)

# Display HTML content
    display(HTML(html_content))
    print()
    display(HTML("<h4 style='color:black; text-align:left;'>Re-run Code cell again for next Prediction</h3>"))

def main_function():

# Set focus to result grid
  set_focus()
  print()
  display(HTML("<h3 style='color:magenta; text-align:center;'>Main Menu - Email ham or spam Prediction</h3>"))
  print()
# Create your buttons for each events
  load_data_button = widgets.Button(description="1. Load Data")
  eda_analysis_button = widgets.Button(description="2. EDA Analysis")
  pre_process_button = widgets.Button(description="3. Pre Process Data")
  transform_build_button = widgets.Button(description="4. Transform & Build")
  load_model_and_predict_button = widgets.Button(description="5. Prediction(Bulk Mails)")
  single_mail_predict_button = widgets.Button(description="6. Prediction(Single Mail)")

# Assign python functions as event handlers
  load_data_button.on_click(load_data)
  eda_analysis_button.on_click(EDA_start)
  pre_process_button.on_click(pre_process_data)
  transform_build_button.on_click(transformation)
  load_model_and_predict_button.on_click(load_model_and_predict)
  single_mail_predict_button.on_click(input_email_and_predict)

# Define custom CSS to adjust button dimensions, backcolor and textcolor

  custom_css = """
  <style>
  .custom-button {
      width: 170px;  /* Change this value for width */
      height: 50px;  /* Change this value for height */
      color: blue; /* Text color */
      background-color: lightgreen;  /* Background color */
      border: 2px solid black;
  }
/* Add a hover effect */
  .custom-button:hover {
      background-color: white; /* Change background color on hover */
      }
  </style>
  """

  # Apply CSS classes to buttons

  load_data_button.add_class("custom-button")
  eda_analysis_button.add_class("custom-button")
  pre_process_button.add_class("custom-button")
  transform_build_button.add_class("custom-button")
  load_model_and_predict_button.add_class("custom-button")
  single_mail_predict_button.add_class("custom-button")

# Display the custom CSS and buttons
# is ensuring that the CSS rules are applied to the buttons' appearance when
# they are displayed in the notebook.

  display(HTML(custom_css))

# Arrange buttons horizontally

  buttons_horizontal_layout_1 = widgets.HBox([load_data_button, eda_analysis_button, pre_process_button])
  buttons_horizontal_layout_2 = widgets.HBox([transform_build_button, load_model_and_predict_button, single_mail_predict_button])

  # Center the buttons using a centered container in result grid

  centered_layout = widgets.Layout(justify_content='center')

  # Arrange buttons horizontally

  centered_container_1 = widgets.Box([buttons_horizontal_layout_1], layout=centered_layout)
  centered_container_2 = widgets.Box([buttons_horizontal_layout_2], layout=centered_layout)

  # Display the centered buttons

  display(centered_container_1)
  print()
  display(centered_container_2)
  print()
# Declare a main function as follows to invoke all other function

if __name__ == "__main__":

  main_function()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Box(children=(HBox(children=(Button(description='1. Load Data', style=ButtonStyle(), _dom_classes=('custom-but…

Box(children=(HBox(children=(Button(description='4. Transform & Build', style=ButtonStyle(), _dom_classes=('cu…